In [1]:
import numpy as np
import gc

gc.collect()
!pip install pickle5
import pickle5 as pickle
from sklearn.decomposition import PCA

You should consider upgrading via the '/home/haseebs/cmput652/bin/python3 -m pip install --upgrade pip' command.


In [2]:
model_name = 'seresnet50'
train_file = 'train_features.pkl'
test_file = 'test_features.pkl'

#model_name = 'resnest50d'
#train_file = 'train_features_resnest50d.pkl'
#test_file = 'test_features_resnest50d.pkl'

image = {
    f'{model_name}_s0':[],
    f'{model_name}_s1':[],
    f'{model_name}_s2':[],
    f'{model_name}_s3':[],
    f'{model_name}_s4':[],
    'label':[]
}
pca1_list=[]
gc.collect()

0

In [3]:
image

{'seresnet50_s0': [],
 'seresnet50_s1': [],
 'seresnet50_s2': [],
 'seresnet50_s3': [],
 'seresnet50_s4': [],
 'label': []}

In [4]:
id = dict()
f = open("../attention-fmri/dataset/image_training_id.csv")
for line in f:
    line = line.strip('\n')
    (key, val) = line.split(",")
    id[val] = key

f = open("../attention-fmri/dataset/image_test_id.csv")
for line in f:
    line = line.strip('\n')
    (key, val) = line.split(",")
    id[val] = key

In [5]:
f = open(f'../attention-fmri/{train_file}', 'rb')
train = pickle.load(f)

f = open(f'../attention-fmri/{test_file}', 'rb')
test = pickle.load(f)

In [6]:
# select random 40k features from all the stages so that we can PCA
#TODO idxes are not consistent
idx = {}
for stage in range(5):
    one_img = list(train[model_name].keys())[0]
    feature_sizes = train[model_name][one_img][stage].flatten().shape
    idx[stage] = np.random.choice(np.arange(feature_sizes[0]), size=40000)

In [7]:
wrong_data=0
for i in range(5):
    for key in train[model_name].keys():
        feat = train[model_name][key][i].flatten()
        image[f'{model_name}_s{i}'].append(feat[idx[i]])
        
    for key in test[model_name].keys():
        feat = test[model_name][key][i].flatten()
        image[f'{model_name}_s{i}'].append(feat[idx[i]])
        
    pca1_list.append(PCA(svd_solver='randomized'))
    image[f'{model_name}_s{i}']=list(pca1_list[-1].fit_transform(np.array(image[f'{model_name}_s{i}'], dtype='float32')))
    print("done stage: ", i)
for key in train[model_name].keys():
    try:
        image['label'].append(id['"' + key + '"'])
    except KeyError:
        print(key)
        wrong_data +=1
        print(wrong_data)
for key in test[model_name].keys():
    try:
        image['label'].append(id['"' + key + '"'])
    except KeyError:
        print(key)
        wrong_data +=1
        print(wrong_data)

for key in image.keys():
    image[key] = np.array(image[key])

done stage:  0
done stage:  1
done stage:  2
done stage:  3
done stage:  4


In [8]:
with open(f'data/pca_features_{model_name}.pkl','wb') as f:
    pickle.dump(image,f)

idx

{0: array([320290, 343979, 754551, ...,  90936, 328603, 156334]),
 1: array([498802, 670331, 449771, ..., 246495, 788080, 245192]),
 2: array([216183, 136623, 376976, ..., 161072, 149580, 387264]),
 3: array([135010,  80134,  20298, ..., 103122,  63368, 195455]),
 4: array([95659, 13870, 63392, ..., 42193, 67937, 28169])}

In [9]:
idx

{0: array([554813,  33426,  65248, ..., 664795, 741973, 257719]),
 1: array([698249, 660178, 727212, ..., 698622, 537273, 290557]),
 2: array([184251, 188872, 350154, ...,  23125,  53738, 208933]),
 3: array([191075,  20429,  14526, ..., 146221, 167767, 171582]),
 4: array([50051, 76048, 80122, ..., 42592, 55592, 57264])}

In [12]:
image['seresnet50_s0'].shape

(1250, 1250)